저장해둔 parquet파일을 읽어온다.
용이한 전처리를 위해 판다스 데이터프레임으로 불러왔다.

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('data/market.parquet')

In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-02 13:29:04,view,12300059,2053013556311359947,construction.tools.drill,bosch,69.17,513196170,00000510-834a-498a-9bed-561a90c5125e
1,2019-11-16 08:11:45,view,1801881,2053013554415534427,electronics.video.tv,samsung,558.55,572115980,000007b4-6d31-4590-b88f-0f68d1cee73c
2,2019-11-01 19:07:11,view,1002544,2053013555631882655,electronics.smartphone,apple,458.25,514001239,000009c4-a1dd-4764-87d9-24f3d7e43c4f
3,2019-11-01 19:08:13,view,1004659,2053013555631882655,electronics.smartphone,samsung,716.76,514001239,000009c4-a1dd-4764-87d9-24f3d7e43c4f
4,2019-11-14 13:16:20,view,1480477,2053013561092866779,computers.desktop,apple,1029.60,529435838,00002268-1b1a-4787-b8ec-e76c2da106aa


Preprocessing

1. category_code컬럼 값 정리('.'기준으로 문자열 나눠 저장)
2. category_id컬럼 삭제
3. event_time컬럼의 datetime 순으로 데이터프레임 정렬

In [4]:
#category_code의 맨 뒤 문자열을 추출하여 product 컬럼으로 추가
df.insert(5,'product', df['category_code'].apply(lambda x : x.split('.')[-1]))

#category_code의 중간 문자열을 추출하여 sub_category 컬럼으로 추가(없는 경우 product값으로 대체됨)
df.insert(5,'sub_category', df['category_code'].apply(lambda x : x.split('.')[1]))

#category_code 값에서 제품명을 제거, '.'대신 '/'으로 교체
df['category_code'] = df['category_code'].apply(lambda x : x.split('.')[0])

#category_id컬럼 삭제
df = df.drop('category_id', axis=1)

In [5]:
#event_time컬럼의 datetime 순으로 데이터프레임 정렬
df.sort_values(by=['event_time'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,event_time,event_type,product_id,category_code,sub_category,product,brand,price,user_id,user_session
0,2019-11-01 00:00:00,view,1003461,electronics,smartphone,smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00,view,5000088,appliances,sewing_machine,sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01,view,1306421,computers,notebook,notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
3,2019-11-01 00:00:01,view,1004775,electronics,smartphone,smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
4,2019-11-01 00:00:01,view,1306894,computers,notebook,notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c


In [7]:
#데이터셋 개요 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42018772 entries, 0 to 42018771
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int32         
 3   category_code  object        
 4   sub_category   object        
 5   product        object        
 6   brand          object        
 7   price          float64       
 8   user_id        int32         
 9   user_session   object        
dtypes: datetime64[ns](1), float64(1), int32(2), object(6)
memory usage: 2.8+ GB


In [8]:
#parquet파일 덮어씌워 저장
df.to_parquet('data/market_final.parquet', compression='gzip')